# Introduction 
This notebooks shows a basic example of how events and the status byte work.



In [1]:
import visa_mock as pyvisa_mock
import time

from pyvisa import ResourceManager
from pyvisa.errors import VisaIOError
from pyvisa.constants import StatusCode

from visa_mock.base.register import register_resource

from geiger_counter_mock import GeigerCounterMock, MessageCode

gc_address = 'MOCK0::mock1::INSTR'
register_resource(gc_address, GeigerCounterMock())

In [2]:
rm = ResourceManager('@mock')
rm.list_resources()
inst = rm.open_resource(gc_address)
print(inst.query('*IDN?'))

geiger counter:fake corp:123456


In [3]:
inst.write('ALERT:EXPOSURE:THRESHOLD 5.01')
print(inst.query('ALERT:EXPOSURE:THRESHOLD?'))
inst.write('ALERT:READING:THRESHOLD 0') # Disable reading threshold

5.01


(25, <StatusCode.success: 0>)

The following block waits for an exposure limit to be hit.  The device triggers a srq when this threshold is reached, which will allow the "wait_for_srq" to return without exception.  In most applications the SRQ should be polled in a seperate thread to allow non-blocking opperation.  In this example we will just wait for the srq to return.

In [10]:
try:
    inst.write('MEAS:EXPOSURE:RESET')
    start_time = time.time()
    inst.write('MEAS:START')

    print('Waiting for exposure limit...')
    try:
        inst.wait_for_srq(timeout=10000) # Wait 10 seconds
    except VisaIOError as e:
        if e.error_code == StatusCode.error_timeout:
            print('No exposure limit reached!')
        else:
            raise e
    else:
        end_time = time.time()
        elapsted_time = end_time - start_time
        stb = inst.stb
        exposure = inst.query("MEAS:EXPOSURE?")
        print(f'SRQ Hit! (stb:{hex(stb)})')
        if stb & MessageCode.READING_LIMIT_EXCEEDED.value:
            print('Reading threshold reached after {elapsted_time} seconds.')
        if stb & MessageCode.EXPOSURE_LIMIT_EXCEEDED.value:
            print(f'Exposure threshold reached after {elapsted_time} seconds. (exposure:{exposure})')
            inst.write('MEAS:EXPOSURE:RESET')
        # Clear the status byte register
        inst.write('*CLS')
finally:
    inst.write('MEAS:STOP')
    print('Done')


Waiting for exposure limit...
SRQ Hit! (stb:0x41)
Exposure threshold reached after 7.11769437789917 seconds. (exposure:5.030765926327754)
Done
